In [215]:
#importing the required libraries
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os


In [216]:
model=models.resnet50(pretrained=True)

In [217]:
return_nodes = {
    'layer1': 'layer1',
    'layer2': 'layer2',
    'layer3': 'layer3',
    'layer4': 'layer4',
}

In [218]:
device=torch.device("cuda")

In [219]:
def image_loader(path):
    image=Image.open(path)
    loader=transforms.Compose([transforms.Resize((128,128)),transforms.ToTensor()])
    image=loader(image).unsqueeze(0)
    return image.to(device,torch.float)

In [220]:
original_image=image_loader('/kaggle/input/human-faces/Humans/1 (1003).jpg')
style_image=image_loader('/kaggle/input/animal-faces/afhq/train/dog/flickr_dog_000031.jpg')
generated_image = torch.nn.Parameter(torch.randn(original_image.shape, requires_grad = True).to("cuda"))

In [221]:
class VGG(nn.Module):
    def __init__(self):
        super(VGG,self).__init__()
        self.req_features= ['1']
        self.model=models.resnet50(pretrained = True).model

    def forward(self,x):
        features = []
        
        feature_extractor = torch.nn.Sequential(*list(self.model.children())[:-1])
        for layer_num,layer in enumerate(feature_extractor):
            x=layer(x)
            if (str(layer_num) in self.req_features):
                features.append(x)

        return features



In [240]:
model=models.resnet50(pretrained = True)

for x in model._modules["layer1"]:
    print(x)


Bottleneck(
  (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (downsample): Sequential(
    (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
Bottleneck(
  (conv1): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fa

In [222]:
class Z(nn.Module):
    def __init__(self):
        
        super(Z,self).__init__()
        self.model = nn.Sequential(
          nn.Conv2d(3,100,1),
          nn.ReLU(),
          nn.Conv2d(100,3,1),
          nn.ReLU()
        )

    def forward(self,x):                 
        return self.model(x)


In [223]:
Zmodel = Z().to("cuda")

In [224]:
def calc_content_loss(gen_feat,orig_feat):
   
    content_l=torch.mean((gen_feat-orig_feat)**2)
    return content_l

In [225]:
def calc_enc_loss(gen_feat,orig_feat):
    content_l=torch.mean((gen_feat-orig_feat)**2)
    return content_l

In [226]:
def calc_style_loss(gen,style):
    batch_size,channel,height,width=gen.size()

    G=torch.mm(gen.view(channel,height*width),gen.view(channel,height*width).t())
    A=torch.mm(style.view(channel,height*width),style.view(channel,height*width).t())
    style_l=torch.mean((G-A)**2)
    return style_l

In [227]:
def calculate_loss(gen_features, orig_feautes, style_featues,):
    style_loss=content_loss=0
    for gen,cont,style in zip(gen_features,orig_feautes,style_featues):

        content_loss+=calc_content_loss(gen,cont)
        style_loss+=calc_style_loss(gen,style)
    
    
    total_loss=alpha*content_loss + beta*(style_loss )
    return total_loss

In [228]:

model=VGG().to(device).eval() 

ModuleAttributeError: 'ResNet' object has no attribute 'model'

In [ ]:
epoch=7000
lr=0.001
alpha=1
beta=1
optimizer=optim.Adam([generated_image],lr=lr)


In [ ]:
subdirs = [[x[0] +y for y in x[2]] for x in os.walk('/kaggle/input/human-faces')][0]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for e in range (epoch):
    gen_features=model(generated_image)
    orig_feautes=model(original_image)
    
    style_featues=model(style_image)
   
    total_loss=calculate_loss(gen_features, orig_feautes, style_featues)
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if(not (e%1000)):
        print(total_loss)
        
        save_image(generated_image,f"gen - {e}.png")
        plt.imshow(generated_image.clone().detach().cpu().numpy()[0].transpose(1,2,0))

In [ ]:
features = []
req_features= ['7']
x = generated_image
feature_extractor = torch.nn.Sequential(*list(model.children()))
for layer_num,layer in enumerate(feature_extractor):    
    x = layer(x)
    print(layer_num,x)
